<a href="https://colab.research.google.com/github/cl16908/models_on_pnr1z/blob/main/EQT_run_pnr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runs the EQT model (Mousavi et al, 2020) on PNR-1z data

#### Cindy Lim Shin Yee
###### Last updated: 21st July 2021
---

The EQTransformer (EQT) model is an encoder-decoder neural network developed by Mousavi et al (2020).
More information, tutorials and documentation about the EQT model can be found here: https://eqtransformer.readthedocs.io/en/latest/index.html

Input:
Seismic dataset (1 hour, 2000 Hz) provided is from the 2018 PNR-1z dataset.

Output:
Event catalogue with event origin times, longitude, latitude, depth, Easting and Northing coordinates.

References:

- Mousavi, S.M., Ellsworth, W.L., Zhu, W., Chuang, L.Y. and Beroza, G.C., 2020. Earthquake transformer—an attentive deep-learning model for simultaneous earthquake detection and phase picking. Nature communications, 11(1), pp.1-12. https://doi.org/10.1038/s41467-020-17591-w

## Installing obspy

In [ ]:
!pip install obspy

In [ ]:
!pip install pyproj

## Please restart the kernel after installing obspy (google colab will not run obspy properly after just installing it once). Go to Runtime > Restart Runtime then run the notebook from the start.

## Importing modules

In [ ]:
import string
import time
import argparse as ap
import sys
import os

import numpy as np
import obspy.core as oc
from obspy.core import Stream
from obspy.core import Trace
from obspy.signal.trigger import trigger_onset
from obspy.core import read
import obspy.signal
import math

import matplotlib as mpl
import pylab as plt
import pandas as pd
from obspy import UTCDateTime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *

import glob
import h5py
import random
import scipy
import pandas as pd
import datetime as datetime
import pyproj


In [ ]:
# For interpolating along well path (needed for locating and orienting stations)
from scipy.interpolate import InterpolatedUnivariateSpline

from tensorflow.python.client import device_lib

## Define functions

In [ ]:
# Define function to produce sliding windows of signal (from original GPD GitHub repo: https://github.com/interseismic/generalized-phase-detection/blob/master/gpd_predict.py)

def sliding_window(data, size, stepsize=1, padded=False, axis=-1, copy=True):
    """
    Calculate a sliding window over a signal
    Parameters
    ----------
    data : numpy array
        The array to be slided over.
    size : int
        The sliding window size
    stepsize : int
        The sliding window stepsize. Defaults to 1.
    axis : int
        The axis to slide over. Defaults to the last axis.
    copy : bool
        Return strided array as copy to avoid sideffects when manipulating the
        output array.
    Returns
    -------
    data : numpy array
        A matrix where row in last dimension consists of one instance
        of the sliding window.
    Notes
    -----
    - Be wary of setting `copy` to `False` as undesired sideffects with the
      output values may occurr.
    Examples
    --------
    >>> a = numpy.array([1, 2, 3, 4, 5])
    >>> sliding_window(a, size=3)
    array([[1, 2, 3],
           [2, 3, 4],
           [3, 4, 5]])
    >>> sliding_window(a, size=3, stepsize=2)
    array([[1, 2, 3],
           [3, 4, 5]])
    See Also
    --------
    pieces : Calculate number of pieces available by sliding
    """
    if axis >= data.ndim:
        raise ValueError(
            "Axis value out of range"
        )

    if stepsize < 1:
        raise ValueError(
            "Stepsize may not be zero or negative"
        )

    if size > data.shape[axis]:
        raise ValueError(
            "Sliding window size may not exceed size of selected axis"
        )

    shape = list(data.shape)
    shape[axis] = np.floor(data.shape[axis] / stepsize - size / stepsize + 1).astype(int)
    shape.append(size)

    strides = list(data.strides)
    strides[axis] *= stepsize
    strides.append(data.strides[axis])

    strided = np.lib.stride_tricks.as_strided(
        data, shape=shape, strides=strides
    )

    if copy:
        return strided.copy()
    else:
        return strided

In [ ]:
# Defining file reading function (from Alan Baird, October 2019)
def readPNR(fname,**kwargs):
    """Wrapper function for reading PNR segy data which sets headers for
    stations and channels automatically"""
    st = read(fname,**kwargs)
    # populate trace headers for network, station name, channel, and distance (depth, needed to easily produce section plots)
    for i, trace in enumerate(st):
        trace.stats.network = "PNR"
        trace.stats.station = "{:0>3d}".format(trace.stats.segy.trace_header.geophone_group_number_of_last_trace)
        trace.stats.channel = 'BS'+str(i%3+1)
        trace.stats.distance = trace.stats.segy.trace_header.receiver_group_elevation/1000
    return st

def readPNR_ENZ(fname,**kwargs):
    """Wrapper function for reading PNR segy data and reorienting it into ENZ
    components witgh appropriate headers.

    Note: Requires that the stns dataframe has been read in correctly"""
    st = readPNR(fname,**kwargs)

    # Create a copy of the traces to be rotated
    strot=st.copy()

    # Loop through stations, construct rotation matrices and apply to apply to traces
    for i, row in stns.iterrows():
        azi=np.radians(row['azi'])
        inc=np.radians(row['inc'])
        rot=np.radians(row['rgb_orient'])

        # Create rotation matrices
        R1=np.array([[np.sin(azi), -np.cos(azi), 0],
                     [np.cos(azi),  np.sin(azi), 0],
                     [          0,            0, 1]])
        R2=np.array([[np.cos(inc), 0, -np.sin(inc)],
                     [          0, 1,            0],
                     [np.sin(inc), 0,  np.cos(inc)]])
        R3=np.array([[ np.cos(rot), np.sin(rot), 0],
                     [-np.sin(rot), np.cos(rot), 0],
                     [           0,           0, 1]])

        # Select traces from appropriate station
        stnm = "{:0>3d}".format(int(row['name']))
        sttmp = strot.select(station=stnm)

        # instrument coordinates are left-handed Hx,Hy,Vz, need to flip polarization of Hy to make it right handed
        tmparrays = np.stack((sttmp[0].data,-sttmp[1].data,sttmp[2].data))

        # Apply the rotation
        rotated = R1 @ R2 @ R3 @ tmparrays

        # Set the data of the traces
        sttmp[0].data=rotated[0,:]
        sttmp[1].data=rotated[1,:]
        sttmp[2].data=rotated[2,:]

        # Rename the channels
        sttmp[0].stats.channel = 'BSE'
        sttmp[1].stats.channel = 'BSN'
        sttmp[2].stats.channel = 'BSZ'
    return strot

## Hyper-parameters

In [ ]:
# Hyperparameters
filter_data = 1
freq_max = 50.0
input_dir = 'Mseeds_rot_filt_18121109'
input_model = 'EqT_model.h5'
stations_json = './PNR_run/PNR.json'
output_dir= 'Detections_rot_filt_18121109_p0.01_bsize20'
try: os.mkdir('./Detections_rot_filt_18121109_p0.01_bsize20/')
except: pass
P_threshold= 0.01
S_threshold= 0.01
overlap= 0.3
batch_size= 20

## Download and load trained EQT model

In [ ]:
!git clone https://github.com/smousavi05/EQTransformer.git

Cloning into 'EQTransformer'...
remote: Enumerating objects: 2030, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 2030 (delta 139), reused 153 (delta 75), pack-reused 1768
Receiving objects: 100% (2030/2030), 51.29 MiB | 27.85 MiB/s, done.
Resolving deltas: 100% (1096/1096), done.


## Get the ground-truth (CMM) catalogue and obtain the continuous data

In [ ]:
!git clone https://github.com/cl16908/PNR_run.git

# Define some paths
drive = './PNR_run/'

# Read the station orientations for trace rotations
# read in event catalogue as a dataframe (note that these don't have traveltime picks)
df_cat = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_FullCatalogue.dat',
                    delim_whitespace=True)

# create a new datetime column with the origin times of the events
df_cat['datetime']=df_cat['T'].apply(UTCDateTime) + df_cat['QC_LOC_T0']

# well path 1
wp1 = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Wellpath.dat', delim_whitespace=True)

# well path 2
wp2 = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-2_Wellpath.dat', delim_whitespace=True)

# location of the injection sleeves of pnr1 and 2
pnr1_stgs = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Stages.dat', delim_whitespace=True)
pnr2_stgs = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-2_Stages.dat', delim_whitespace=True)

# station locations and "orientations"
stns = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Stations_orient.dat', delim_whitespace=True)


# Functions to return well inclination and azimuth from measured depth
md2inc=InterpolatedUnivariateSpline(wp2['MD'],wp2['Inc'])
md2az=InterpolatedUnivariateSpline(wp2['MD'],wp2['Az'])


# Find measured depth for each station by root finding
MDs=[]
for stdp in stns['stz']:
    MDs.append(InterpolatedUnivariateSpline(wp2['MD'],wp2['TVD_MSL']-stdp).roots()[0])

# assign station measured depth and compute station azimuth and inclination
stns['MD'] = MDs
stns['azi']=stns['MD'].apply(md2az)
stns['inc']=stns['MD'].apply(md2inc)

Cloning into 'PNR_run'...
remote: Enumerating objects: 283, done.
remote: Total 283 (delta 0), reused 0 (delta 0), pack-reused 283
Receiving objects: 100% (283/283), 822.98 MiB | 22.44 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (241/241), done.


In [ ]:
# Reading filenames of the segy files

this_folder = '181211'
hour = '09'
filenames1 = int('20'+this_folder + hour)
filenames = drive + '/data/' + this_folder[0:6] + '/' + str(filenames1) + '*.segy'
fhour = glob.glob(filenames)
fhour.sort()

In [ ]:
# PRE-ALLOCATE instead
ahour = np.zeros((24,2000*16*len(fhour),3))

try:
  for j in np.arange(0,len(fhour)):
      sthour = readPNR_ENZ(fhour[j],unpack_trace_headers = True)
      if j ==0:
          firststarttime = sthour[0].stats.starttime
      if j==len(fhour)-1:
          lastendtime = sthour[0].stats.endtime
      for i in np.arange(0,len(sthour)):
          ahour[math.floor(i/3),np.arange(j*2000*16,(j+1)*2000*16),(i%3)] = sthour[i].data
except:
  !pip uninstall -y obspy
  !pip install obspy # reinstalling obspy -> click on the restart runtime button and re-run the whole notebook
  print('This happens because obspy is not reading in the data. The only fix I have found for this is restarting the runtime and re-running the notebook. Should work after 2-3 tries!!')
  print("***Please restart runtime (Click on Restart Runtime in this cell) and re-run notebook from beginning (Runtime -> Run All)***")

# TIME MATRIX TOTAL
t = np.arange(0,len(fhour)*16,0.0005) # from 0 to 3600 seconds, 0.0005 interval
st = sthour
a = ahour
dt = 0.0005

stations = list(set([st[s].stats.station for s in np.arange(0, len(st))]))
stations.sort()

# INPUT FIRST FILE NUMBER NAME
name = fhour[0]

# set starttime
for i in np.arange(0,len(st)):
    st[i].stats.starttime = UTCDateTime(int(name[-23:-19]), int(name[-19:-17]), int(name[-17:-15]), int(name[-15:-13]), int(name[-13:-11]), int(name[-11:-9]))

for s in np.arange(0,len(stations)):
    st2 = st.select(station=stations[s])

chan = st2[0].stats.channel
sr = st2[0].stats.sampling_rate
dt = st2[0].stats.delta
net = st2[0].stats.network
sta = st2[0].stats.station

# Detrending
for i in np.arange(0,(ahour.shape[0])-1):
    a[i,:,0] = obspy.signal.detrend.simple(a[i,:,0])
    a[i,:,1] = obspy.signal.detrend.simple(a[i,:,1])
    a[i,:,2] = obspy.signal.detrend.simple(a[i,:,2])
    

In [ ]:
# filter data matrix
if filter_data:
    import obspy.signal.filter as obs
    for i in np.arange(0,(ahour.shape[0])-1):
        a[i,:,0] = obs.highpass(a[i,:,0],freq=freq_max,df=st[0].stats.sampling_rate)
        a[i,:,1] = obs.highpass(a[i,:,1], freq=freq_max, df=st[0].stats.sampling_rate)
        a[i,:,2] = obs.highpass(a[i,:,2], freq=freq_max, df=st[0].stats.sampling_rate)

## Convert continuous data into mseed format (EQT input)

In [ ]:
try: os.mkdir('./Mseeds_rot_filt_18121109')
except: pass

try:
  # make a folder for each station
  for i in np.arange(0,24):
      if i >= 9:
          os.mkdir('./Mseeds_rot_filt_18121109/PR0' + str(i+1))
      else:
          os.mkdir('./Mseeds_rot_filt_18121109/PR00' + str(i+1))
except: pass


In [ ]:
# OUTPUT: 72 MSEED traces= 24 stations, 3 components for 1 minute data
firststarttime = sthour[0].stats.starttime
for i in np.arange(0,len(sthour)):
    stats = {'network': sthour[i].stats.network, 'station': sthour[i].stats.station, 'starttime': firststarttime, 'endtime': lastendtime,
             'location': '', 'channel': sthour[i].stats.channel, 'npts': a.shape[1], 'sampling_rate': 2000, 'delta': 1/2000,
         'mseed': {'dataquality': 'D'},
         'distance': sthour[i].stats.distance, 'segy': sthour[i].stats.segy}
    st = Stream([Trace(data=a[i//3,:,i%3],header=stats)])
    # write as ASCII file (encoding=0)
    if (i//3)+1 >= 10:
        st.write("./Mseeds_rot_filt_18121109/PR0"+ str((i//3)+1) + "/PR.PR0"+ str((i//3)+1) + ".." + str(stats['channel']) + "__" + str(firststarttime)[0:13] + str(firststarttime)[14:16] + str(firststarttime)[17:19] + "Z" + "__" + str(lastendtime)[0:13] + str(lastendtime)[14:16] + str(lastendtime)[17:19] + "Z" +".mseed", format='MSEED')
    else:
        st.write("./Mseeds_rot_filt_18121109/PR00"+ str((i//3)+1) + "/PR.PR00"+ str((i//3)+1) + ".." + str(stats['channel']) + "__" + str(firststarttime)[0:13] + str(firststarttime)[14:16] + str(firststarttime)[17:19] + "Z" +"__" + str(lastendtime)[0:13] + str(lastendtime)[14:16] + str(lastendtime)[17:19] + "Z" + ".mseed", format='MSEED')
    

/usr/local/lib/python3.7/dist-packages/obspy/io/mseed/core.py:715: UserWarning: Detected non contiguous data array in Stream[0].data. Trying to fix array.
  warnings.warn(msg)


# Running the EQT model on the catalogue



In [ ]:
# Copy relevant EQT scripts to run the model
!cp EQTransformer/EQTransformer/core/EqT_utils.py .
!cp PNR_run/mseed_predictor_colab.py .
!cp EQTransformer/ModelsAndSampleData/EqT_model.h5 .

from EqT_utils import f1, SeqSelfAttention, FeedForward, LayerNormalization
from mseed_predictor_colab import mseed_predictor

time1 = datetime.datetime.now()

# Run EQT mseed_predictor - type 'y' to create a new directory for the results!
mseed_predictor(input_dir=input_dir,input_model=input_model,stations_json=stations_json,output_dir=output_dir,P_threshold=P_threshold,S_threshold=S_threshold,number_of_plots = 100,plot_mode='time_frequency',overlap=overlap,batch_size=batch_size)

time2 = datetime.datetime.now()

print('EQTransformer took',time2-time1,'on 1 hour of data for 24 stations.')

08-27 16:10 [INFO] [EQTransformer] Running EqTransformer  0.1.61
08-27 16:10 [INFO] [EQTransformer] *** Loading the model ...
08-27 16:10 [DEBUG] [h5py._conv] Creating converter from 3 to 5
08-27 16:11 [INFO] [EQTransformer] *** Loading is complete!
08-27 16:11 [INFO] [EQTransformer] *** /content/Detections_rot_filt_18121109_p0.01_bsize20 already exists!


 --> Type (Yes or y) to create a new empty directory! This will erase your previous results so make a copy if you want them.y


08-27 16:13 [INFO] [EQTransformer] There are files for 24 stations in Mseeds_rot_filt_18121109 directory.
08-27 16:13 [INFO] [EQTransformer] Started working on PR001, 1 out of 24 ...
08-27 16:13 [INFO] [EQTransformer] 2018-12-11T090003Z__2018-12-11T100002Z.mseed
08-27 16:13 [DEBUG] [matplotlib.font_manager] findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=12.0.
08-27 16:13 [DEBUG] [matplotlib.font_manager] findfont: score(<Font 'DejaVu Sans' (DejaVuSans-Oblique.ttf) oblique normal 400 normal>) = 1.05
08-27 16:13 [DEBUG] [matplotlib.font_manager] findfont: score(<Font 'DejaVu Serif' (DejaVuSerif-Bold.ttf) normal normal 700 normal>) = 10.335
08-27 16:13 [DEBUG] [matplotlib.font_manager] findfont: score(<Font 'STIXSizeOneSym' (STIXSizOneSymBol.ttf) normal normal 700 normal>) = 10.335
08-27 16:13 [DEBUG] [matplotlib.font_manager] findfont: score(<Font 'STIXSizeOneSym' (STIXSizOneSymReg.ttf) normal normal regular normal>) = 10.05
08-27 16:13 [DEBU

08-27 16:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 23.32 seconds.
08-27 16:13 [INFO] [EQTransformer] *** Detected: 14 events.
08-27 16:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /content/Detections_rot_filt_18121109_p0.01_bsize20/PR001_outputs "
08-27 16:13 [INFO] [EQTransformer] Started working on PR002, 2 out of 24 ...
08-27 16:13 [INFO] [EQTransformer] 2018-12-11T090003Z__2018-12-11T100002Z.mseed


08-27 16:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 10.0 seconds.
08-27 16:13 [INFO] [EQTransformer] *** Detected: 3 events.
08-27 16:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /content/Detections_rot_filt_18121109_p0.01_bsize20/PR002_outputs "
08-27 16:13 [INFO] [EQTransformer] Started working on PR003, 3 out of 24 ...
08-27 16:13 [INFO] [EQTransformer] 2018-12-11T090003Z__2018-12-11T100002Z.mseed


...

08-27 16:17 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 10.75 seconds.
08-27 16:17 [INFO] [EQTransformer] *** Detected: 3 events.
08-27 16:17 [INFO] [EQTransformer]  *** Wrote the results into --> " /content/Detections_rot_filt_18121109_p0.01_bsize20/PR021_outputs "
08-27 16:17 [INFO] [EQTransformer] Started working on PR022, 22 out of 24 ...
08-27 16:17 [INFO] [EQTransformer] 2018-12-11T090003Z__2018-12-11T100002Z.mseed


08-27 16:17 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 9.09 seconds.
08-27 16:17 [INFO] [EQTransformer] *** Detected: 2 events.
08-27 16:17 [INFO] [EQTransformer]  *** Wrote the results into --> " /content/Detections_rot_filt_18121109_p0.01_bsize20/PR022_outputs "
08-27 16:17 [INFO] [EQTransformer] Started working on PR023, 23 out of 24 ...
08-27 16:17 [INFO] [EQTransformer] 2018-12-11T090003Z__2018-12-11T100002Z.mseed


08-27 16:17 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 12.36 seconds.
08-27 16:17 [INFO] [EQTransformer] *** Detected: 8 events.
08-27 16:17 [INFO] [EQTransformer]  *** Wrote the results into --> " /content/Detections_rot_filt_18121109_p0.01_bsize20/PR023_outputs "
08-27 16:17 [INFO] [EQTransformer] Started working on PR024, 24 out of 24 ...
08-27 16:17 [INFO] [EQTransformer] 2018-12-11T090003Z__2018-12-11T100002Z.mseed


08-27 16:17 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 8.86 seconds.
08-27 16:17 [INFO] [EQTransformer] *** Detected: 1 events.
08-27 16:17 [INFO] [EQTransformer]  *** Wrote the results into --> " /content/Detections_rot_filt_18121109_p0.01_bsize20/PR024_outputs "


EQTransformer took 0:06:41.578345 on 1 hour of data for 24 stations.


## Model output => input for phase association

In [ ]:
folder = './Detections_rot_filt_18121109_p0.01_bsize20/'
stn_folder = glob.glob(folder + '*')
stn_folder.sort()

In [ ]:
for i in np.arange(0,len(stn_folder)):
    if i == 0:
        picks_df = pd.read_csv(glob.glob(stn_folder[i]+'/X_prediction_results.csv')[0],usecols=['station','p_arrival_time','p_probability','s_arrival_time','s_probability'])
    else:
        picks_df2 = pd.read_csv(glob.glob(stn_folder[i]+'/X_prediction_results.csv')[0],usecols=['station','p_arrival_time','p_probability','s_arrival_time','s_probability'])
        picks_df = pd.concat([picks_df,picks_df2])

In [ ]:
picks_df

,station,p_arrival_time,p_probability,s_arrival_time,s_probability
0,PR001,2018-12-11 09:21:37.620000,0.73,2018-12-11 09:21:37.860000,0.34
1,PR001,2018-12-11 09:26:49.930000,0.82,2018-12-11 09:26:50.680000,0.66
2,PR001,2018-12-11 09:27:05.310000,0.15,2018-12-11 09:27:05.950000,0.24
3,PR001,2018-12-11 09:28:28.730000,0.52,2018-12-11 09:28:28.880000,0.82
4,PR001,2018-12-11 09:30:29.560000,0.34,2018-12-11 09:30:29.840000,0.42
...,...,...,...,...,...
4,PR023,2018-12-11 09:44:39.840000,0.85,2018-12-11 09:44:41.030000,0.44
5,PR023,2018-12-11 09:47:44.570000,0.16,2018-12-11 09:47:47.430000,0.64
6,PR023,2018-12-11 09:50:01.830000,0.57,2018-12-11 09:50:04.960000,0.72
7,PR023,2018-12-11 09:53:31.450000,0.79,2018-12-11 09:53:31.670000,0.35


In [ ]:
# get station name into correct format

for i in np.arange(0,len(picks_df)):
    picks_df['station'].iloc[i] =  picks_df['station'].iloc[i][0:2] + picks_df['station'].iloc[i][3:]

In [ ]:
NLL_df = pd.DataFrame(columns=['time','sta','instrument','comp','p_onset','phase','prob','first_motion','date (yyyymmdd)','hhmm','ss','err','errmag','coda','amplitude','period'])
NLL_df

,time,sta,instrument,comp,p_onset,phase,prob,first_motion,date (yyyymmdd),hhmm,ss,err,errmag,coda,amplitude,period


In [ ]:
## counting how many entries there should be in the NLL_df

count_i = 0
phases = []
stn_list = []
time_list = []
probs = []

for i in np.arange(0,len(picks_df)):
    
    # check if there is a p arrival time
    if picks_df['p_arrival_time'].isnull().values[i]: # if it's null, pass
        pass
    else:
        count_i += 1
        phases.append('p')
        stn_list.append(picks_df['station'].iloc[i])
        time_list.append(picks_df['p_arrival_time'].iloc[i])
        probs.append(picks_df['p_probability'].iloc[i])
        
    # check if there is an s arrival time
    if picks_df['s_arrival_time'].isnull().values[i]: # if it's null, pass
        pass
    else:
        count_i += 1
        phases.append('s')
        stn_list.append(picks_df['station'].iloc[i])
        time_list.append(picks_df['s_arrival_time'].iloc[i])
        probs.append(picks_df['s_probability'].iloc[i])

In [ ]:
prob_rank = []
errmag = []
for i in np.arange(0,len(probs)):
    if probs[i] >= 0.85:
        prob_rank.append(0)
        errmag.append(5*1/2000)

    elif probs[i] >=0.7:
        prob_rank.append(1)
        errmag.append(10*1/2000)

    elif probs[i] >= 0.6:
        prob_rank.append(2)
        errmag.append(20*1/2000)

    elif probs[i] >= 0.5:
        prob_rank.append(3)
        errmag.append(50*1/2000)

    elif probs[i] < 0.5:
        prob_rank.append(4)
        errmag.append(99999.9)

    elif math.isnan(probs[i]):
        prob_rank.append(np.NaN)
        errmag.append(np.NaN)

In [ ]:
NLL_df['instrument'] = ['?'] * count_i
NLL_df['comp'] = ['?'] * count_i
NLL_df['p_onset'] = ['?'] * count_i
NLL_df['first_motion'] = ['?'] * count_i
NLL_df['err'] = ['GAU'] * count_i
NLL_df['coda'] = [-1.0] * count_i
NLL_df['amplitude'] = [-1.0] * count_i
NLL_df['period'] = [-1.0] * count_i

for i in np.arange(0,count_i):
    
    NLL_df['sta'].iloc[i] = stn_list[i]
    
    NLL_df['phase'].iloc[i] = phases[i]
    
    NLL_df['time'].iloc[i] = time_list[i]
    # date
    NLL_df['date (yyyymmdd)'].iloc[i] = time_list[i][0:4]+time_list[i][5:7]+time_list[i][8:10]
    # hhmm
    NLL_df['hhmm'].iloc[i] = time_list[i][11:13]+ time_list[i][14:16]
    # ss
    NLL_df['ss'].iloc[i] = time_list[i][17:17+7]
    
    NLL_df['errmag'].iloc[i] = errmag[i]
    
    NLL_df['prob'].iloc[i] = probs[i]

for i in np.arange(0,len(NLL_df)):
    if len(NLL_df['ss'].iloc[i]) < 7:
        NLL_df['ss'].iloc[i] = NLL_df['ss'].iloc[i] + '.0000'
    else:
        pass

In [ ]:
NLL_df_time = NLL_df.sort_values(by='time')

In [ ]:
NLL_df_time

,time,sta,instrument,comp,p_onset,phase,prob,first_motion,date (yyyymmdd),hhmm,ss,err,errmag,coda,amplitude,period
178,2018-12-11 09:19:19.320000,PR20,?,?,?,p,0.26,?,20181211,0919,19.3200,GAU,99999.9,-1.0,-1.0,-1.0
179,2018-12-11 09:19:22.320000,PR20,?,?,?,s,0.79,?,20181211,0919,22.3200,GAU,0.005,-1.0,-1.0,-1.0
180,2018-12-11 09:19:43.040000,PR20,?,?,?,p,0.09,?,20181211,0919,43.0400,GAU,99999.9,-1.0,-1.0,-1.0
181,2018-12-11 09:19:44.860000,PR20,?,?,?,s,0.57,?,20181211,0919,44.8600,GAU,0.025,-1.0,-1.0,-1.0
182,2018-12-11 09:20:34.970000,PR20,?,?,?,p,0.57,?,20181211,0920,34.9700,GAU,0.025,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2018-12-11 09:54:25.780000,PR20,?,?,?,s,0.49,?,20181211,0954,25.7800,GAU,99999.9,-1.0,-1.0,-1.0
136,2018-12-11 09:54:37.580000,PR15,?,?,?,p,0.83,?,20181211,0954,37.5800,GAU,0.005,-1.0,-1.0,-1.0
207,2018-12-11 09:54:37.760000,PR20,?,?,?,p,0.28,?,20181211,0954,37.7600,GAU,99999.9,-1.0,-1.0,-1.0
137,2018-12-11 09:54:39.210000,PR15,?,?,?,s,0.67,?,20181211,0954,39.2100,GAU,0.01,-1.0,-1.0,-1.0


# Phase Association for event location

In [ ]:
###############################
# Grouping Ps
###############################
print('Grouping Ps.')
df = NLL_df_time
# Dataframe with only Ps
df_P = df[df['phase']=='p']
df_P = df_P.reset_index(drop=True)

# convert all string 'time' entries to UTCDateTime

for i in np.arange(0,len(df_P)):
    df_P['time'].values[i] = UTCDateTime(df_P['time'].values[i])
    
timewin = 0.2
Pevs = pd.DataFrame()
j = 1
group = pd.DataFrame()
ngroup = 1
for i in np.arange(0,len(df_P)):
    group_add = pd.DataFrame()
    # if the absolute diff is <=0.2s, it will be added to the group
    if (abs(df_P['time'].iloc[i]-df_P['time'].iloc[j]) <= 0.2):
        group_add = df_P.iloc[i:i+1]
        group = pd.concat([group,group_add])
        
    else:
        pass
    
        if len(group)>= 4:
            group['group_no'] = [ngroup] * len(group)
            # concatenate to Pevs + add a null column
            null_col = pd.DataFrame([[np.nan] * len(group.columns)],columns=group.columns)
            Pevs = pd.concat([Pevs,group])
#             Pevs = pd.concat([Pevs,null_col])
            ngroup += 1
        else:
            
            pass
        
        j = i
        # set group to nothing again
        group = pd.DataFrame()
        group_add = df_P.iloc[i:i+1]
        group = pd.concat([group,group_add])
        
# out of loop, save the last group
j = i
if len(group)>= 4:
    group['group_no'] = [ngroup] * len(group)
    Pevs = pd.concat([Pevs, group])
    ngroup += 1

Pevs = Pevs.reset_index(drop=True)

# check if last row is null
if pd.isnull(Pevs['err'].iloc[-1]):
    Pevs = Pevs[:-1]


Grouping Ps.


In [ ]:
###############################
# Grouping Ss
###############################
print('Grouping Ss.')
df = NLL_df_time
# Dataframe with only Ss
df_S = df[df['phase']=='s']
df_S = df_S.reset_index(drop=True)

# convert all string 'time' entries to UTCDateTime

for i in np.arange(0,len(df_S)):
    df_S['time'].values[i] = UTCDateTime(df_S['time'].values[i])
    
timewin = 0.25
Sevs = pd.DataFrame()
j = 1
group = pd.DataFrame()
ngroup = 1
for i in np.arange(0,len(df_S)):
    group_add = pd.DataFrame()
    # if the absolute diff is <=0.2s, it will be added to the group
    if (abs(df_S['time'].iloc[i]-df_S['time'].iloc[j]) <= 0.2):
        group_add = df_S.iloc[i:i+1]
        group = pd.concat([group,group_add])
        
    else:
        pass
    
        if len(group)>= 4:
            group['group_no'] = [ngroup] * len(group)
            # concatenate to Pevs + add a null column
            null_col = pd.DataFrame([[np.nan] * len(group.columns)],columns=group.columns)
            Sevs = pd.concat([Sevs,group])
            Sevs = pd.concat([Sevs,null_col])
            ngroup += 1
        else:
            
            pass
        
        j = i
        # set group to nothing again
        group = pd.DataFrame()
        group_add = df_S.iloc[i:i+1]
        group = pd.concat([group,group_add])
        
# out of loop, save the last group
j = i
if len(group)>= 4:
    group['group_no'] = [ngroup] * len(group)
    Sevs = pd.concat([Sevs, group])
    ngroup += 1

Sevs = Sevs.reset_index(drop=True)

# check if last row is null
if pd.isnull(Sevs['err'].iloc[-1]):
    Sevs = Sevs[:-1]


Grouping Ss.


In [ ]:
# associating S to the Ps
# if Stime between Ptime and Ptime + 0.13
# if  Ptime <= Stime <= Ptime + 0.13
print('Phase associating.')
PSevs = Pevs.copy()
PSevs['stime'] = np.nan * len(PSevs)
Stime = []
# add Pevs['prob_s']
PSevs['prob_s'] = np.nan * len(PSevs)
# PSevs reset index
PSevs = PSevs.reset_index(drop=True)

# for all P phase entries, search through the S arrival picks
for j in np.arange(0,len(PSevs)):
    Stime = []
    # same station in df_S dataframe
    samesta_df = df_S[df_S['sta']==PSevs['sta'].iloc[j]]
    samesta_df = samesta_df.reset_index(drop=True)

    for i in np.arange(0,len(samesta_df)):
        # if within time window, add to Stime matrix
        if (samesta_df['time'].iloc[i] <= PSevs['time'].iloc[j] + 0.13) and (samesta_df['time'].iloc[i] >= PSevs['time'].iloc[j]):
            Stime.append(samesta_df['time'].iloc[i])
            
            # if S picks have multiple times, take the higher prob_S
            if len(Stime)>1:
                # if new Sevs prob > prob already set, overwrite:
                if samesta_df['prob'].iloc[i] > PSevs['prob_s'].iloc[j]:
                    PSevs['stime'].iloc[j] = samesta_df['time'].iloc[i]
                    PSevs['prob_s'].iloc[j] = samesta_df['prob'].iloc[i]
                    # print('added! x2')        

                else:
                    pass
            # else add to PSevs dataframe
            else:
            
                PSevs['stime'].iloc[j] = samesta_df['time'].iloc[i]
                PSevs['prob_s'].iloc[j] = samesta_df['prob'].iloc[i]

    else:
        pass          


PSevs['pha_s'] = np.nan * len(PSevs)
for i in np.arange(0,len(PSevs)):
    if pd.isnull(PSevs['stime'].iloc[i]):
        pass
    else:
        PSevs['pha_s'].iloc[i] = 's'

# resetting index
PSevs = PSevs.reset_index(drop=True)

Phase associating.


In [ ]:
# add a null space after every Stime
idxnan = []
for i in np.arange(0,len(PSevs)):
    if pd.isnull(PSevs['stime'].iloc[i]):
        pass
    else:
        idxnan.append(i)

for i in np.arange(0,len(idxnan)):
    idxnan[i] = idxnan[i] + .5

idxnew = PSevs.index.union(idxnan)[:-1]
PSevs = PSevs.reindex(idxnew).reset_index(drop=True)

# if time value is nan, fill in with details from before
for i in np.arange(1,len(PSevs)):
    
    if pd.isnull(PSevs['time'].iloc[i]):
        PSevs['time'].values[i] = PSevs['stime'].values[i-1]
        PSevs['sta'].values[i] = PSevs['sta'].values[i-1]
        PSevs['phase'].values[i] = PSevs['pha_s'].values[i-1]
        PSevs['prob'].values[i] = PSevs['prob_s'].values[i-1]
        PSevs['group_no'].values[i] = PSevs['group_no'].values[i-1]
    else:
        pass

# empty space after every group no
idx = PSevs.index.union(PSevs.index[PSevs['group_no'].shift(-1).ne(PSevs['group_no'])] + .5)[:-1]

PSevs = PSevs.reindex(idx).reset_index(drop=True)

# dropping all unnecessary columns
PSevs = PSevs.drop(columns=['stime','prob_s','pha_s','group_no'])

cols = ['sta','phase','prob','time']

PSevs = PSevs[cols]

In [ ]:
PSevs

,sta,phase,prob,time
0,PR17,p,0.69,2018-12-11T09:21:37.420000Z
1,PR02,p,0.24,2018-12-11T09:21:37.460000Z
2,PR14,p,0.77,2018-12-11T09:21:37.460000Z
3,PR09,p,0.13,2018-12-11T09:21:37.470000Z
4,PR03,p,0.29,2018-12-11T09:21:37.580000Z
5,PR07,p,0.52,2018-12-11T09:21:37.590000Z
6,PR01,p,0.73,2018-12-11T09:21:37.620000Z
7,NaN,NaN,NaN,NaN
8,PR04,p,0.61,2018-12-11T09:26:49.860000Z
9,PR16,p,0.87,2018-12-11T09:26:49.890000Z


In [ ]:
##################################
# OBSERVATION FILE MAKER
##################################

# new dataframe for nlloc
PSobs = pd.DataFrame(columns=['sta','instrument','comp','Ponset','phase','firstmotion','date(yyyymmdd)','hhmm','ss','err','errmag','coda_duration','amplitude','period','priorwt'])

# convert all UTCDateTime times to string
for i in np.arange(0,len(PSevs)):

    PSevs['time'].values[i] = str(PSevs['time'].values[i])

#     if isinstance(PSevs['time'].values[i],str):
#         PSevs['time'].values[i] = PSevs['time'].values[i][2:4] + PSevs['time'].values[i][5:7] + PSevs['time'].values[i][8:10] + PSevs['time'].values[i][11:13] + PSevs['time'].values[i][14:16] + PSevs['time'].values[i][17:25]
#         # PSevs['time'].values[i] = '{:07.4f}'.format(PSevs['time'].values[i])
#     else:
#         pass

PSobs['sta'] = PSevs['sta']
PSobs['date(yyyymmdd)'] = PSevs['time']
PSobs['hhmm'] = PSevs['time']
PSobs['ss'] = PSevs['time']

# defining station, instrument, comp, Ponset, phase, firstmotion, date, err, coda_duration, amplitude and period
for i in np.arange(0,len(PSevs)):
    
    if PSobs['sta'].values[i] == str(PSobs['sta'].values[i]):

        PSobs['sta'].values[i] = PSevs['sta'].values[i][0:4]
        
        PSobs['instrument'].values[i] = '?'
        PSobs['comp'].values[i] = '?'
        PSobs['Ponset'].values[i] = '?'
        PSobs['phase'].values[i] = PSevs['phase'].values[i]
        PSobs['firstmotion'].values[i] = '?'
        PSobs['date(yyyymmdd)'].values[i] = PSevs['time'].values[i][0:4]+PSevs['time'].values[i][5:7]+PSevs['time'].values[i][8:10]
        PSobs['hhmm'].values[i] = PSevs['time'].values[i][11:13]+ PSevs['time'].values[i][14:16]
        PSobs['ss'].values[i] = PSevs['time'].values[i][17:17+7]
        PSobs['err'].values[i] = 'GAU'
        PSobs['errmag'].values[i] = '?'
        PSobs['coda_duration'].values[i] = -1.00e+00
        PSobs['amplitude'].values[i] =-1.00e+00
        PSobs['period'].values[i] =-1.00e+00  
        
    else:
        pass
    
# defining probability ranking and errmag columns
PSobs['prob'] = PSevs['prob']
PSobs['prob_rank']= np.arange(0,len(PSobs))
prob_rank = []
errmag = []
for i in np.arange(0,len(PSevs)):

    if PSobs['prob'].values[i] >= 0.85:
        prob_rank.append(0)
        errmag.append(5*1/2000)

    elif PSobs['prob'].values[i] >=0.7:
        prob_rank.append(1)
        errmag.append(10*1/2000)
        
    elif PSobs['prob'].values[i] >= 0.6:
        prob_rank.append(2)
        errmag.append(20*1/2000)

    elif PSobs['prob'].values[i] >= 0.5:
        prob_rank.append(3)
        errmag.append(50*1/2000)
        
    elif PSobs['prob'].values[i] < 0.5:
        prob_rank.append(4)
        errmag.append(99999.9)
        
    elif math.isnan(PSobs['prob'].values[i]):
        prob_rank.append(np.NaN)
        errmag.append(np.NaN)
PSobs['prob_rank'] = prob_rank
PSobs['errmag'] = errmag


In [ ]:
for i in np.arange(0,len(PSobs)):
    if not isinstance(PSobs['sta'].iloc[i],str):
        PSobs['date(yyyymmdd)'].iloc[i] = np.nan
        PSobs['hhmm'].iloc[i] = np.nan
        PSobs['ss'].iloc[i] = np.nan
    else:
        pass

print('Saving hpf file.')
print('check which drive/location of save.')
PSobs[['sta','instrument','comp','Ponset','phase','firstmotion','date(yyyymmdd)','hhmm','ss','err','errmag','coda_duration','amplitude','period']].to_csv(drive +  'EQT_testNLLOC_OBS.hpf', header=None, index=None, sep=' ')

Saving hpf file.
check which drive/location of save.


# NLLOC - event location

In [ ]:
######################
# NLLOC 
######################

# download NLLoc
!wget http://alomax.free.fr/nlloc/soft7.00/tar/NLL7.00_src.tgz; tar -zxf NLL7.00_src.tgz; cd src; make distrib

# make directories

!mkdir ./NLLoc
!mkdir ./NLLoc/picks
!mkdir ./NLLoc/model
!mkdir ./NLLoc/time
!mkdir ./NLLoc/loc

--2021-09-11 00:22:19--  http://alomax.free.fr/nlloc/soft7.00/tar/NLL7.00_src.tgz
Resolving alomax.free.fr (alomax.free.fr)... 212.27.63.116
Connecting to alomax.free.fr (alomax.free.fr)|212.27.63.116|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407847 (398K) [application/x-gzip]
Saving to: ‘NLL7.00_src.tgz’

NLL7.00_src.tgz     100%[===================>] 398.29K   909KB/s    in 0.4s    

2021-09-11 00:22:20 (909 KB/s) - ‘NLL7.00_src.tgz’ saved [407847/407847]

gcc -c -O3 -Wall -std=gnu99  NLLoc_main.c  
gcc -c -O3 -Wall -std=gnu99  NLLoc1.c  
NLLoc1.c: In function ‘NLLoc’:
NLLoc1.c:277:9: warning: ignoring return value of ‘system’, declared with attribute warn_unused_result [-Wunused-result]
         system(sys_command);
         ^~~~~~~~~~~~~~~~~~~
NLLoc1.c:280:9: warning: ignoring return value of ‘system’, declared with attribute warn_unused_result [-Wunused-result]
         system(sys_command);
         ^~~~~~~~~~~~~~~~~~~
NLLoc1.c:612:17: warning: ignor

In [ ]:
# blanketPScindy_nll.in
! ./src/Vel2Grid ./PNR_run/blanketPScindy_nll.in

Vel2Grid (NonLinLoc v7.00.00 27Oct2017) 
CONTROL:  MessageFlag: 3  RandomNumSeed: 54321
TRANSFORM  SIMPLE LatOrig 53.775770  LongOrig -2.987780  RotCW 0.000000
Vel2Grid files:  Output: ./NLLoc/model/3km0.025.*
Vel2Grid wave type:  P
Vel2Grid wave type:  S
GRID: {x, y, z}
  Num: {121, 121, 121}
  Orig: {0, 0, 0}
  LenSide: {0.025, 0.025, 0.025}
  Type: SLOW_LEN
Creating model grid files: ./NLLoc/model/3km0.025.P.mod.*
Creating model grid files: ./NLLoc/model/3km0.025.S.mod.*


In [ ]:
# Pg2tblanketPScindy_nll.in
! ./src/Grid2Time ./PNR_run/Pg2tblanketPScindy_nll.in

# Sg2tblanketPScindy_nll.in
! ./src/Grid2Time ./PNR_run/Sg2tblanketPScindy_nll.in

Streaming output truncated to the last 5000 lines.
update side z=110: ff fb bb bf z=110 <R#1>updated.
update side z=109: ff fb bb bf z=109 <R#1>updated.
update side z=108: ff fb bb bf z=108 <R#1>updated.
update side z=107: ff fb bb bf z=107 <R#1>updated.
update side z=106: ff fb bb bf z=106 <R#1>updated.
update side z=105: ff fb bb bf z=105 <R#1>updated.
update side z=104: ff fb bb bf z=104 <R#1>updated.
update side z=103: ff fb bb bf z=103 <R#1>updated.
update side z=102: ff fb bb bf z=102 <R#1>updated.
update side z=101: ff fb bb bf z=101 <R#1>updated.
update side z=100: ff fb bb bf z=100 <R#1>updated.
update side z=99: ff fb bb bf z=99 <R#1>updated.
update side z=98: ff fb bb bf z=98 <R#1>updated.
update side z=97: ff fb bb bf z=97 <R#1>updated.
update side z=96: ff fb bb bf z=96 <R#1>updated.
update side z=95: ff fb bb bf z=95 <R#1>updated.
update side z=94: ff fb bb bf z=94 <R#1>updated.
update side z=93: ff fb bb bf z=93 <R#1>updated.
update side z=92: ff fb bb bf z=92 <R#1>updat

In [ ]:
# NLLOC from code before
os.system('sed "s|LOCFILES ./NLLOC_OBS/PS_181015_NLLOC_OBS.hpf NLLOC_OBS ./NLLOC/time/3km0.025 ./NLLoc/loc/181015/ALLPS|LOCFILES ./PNR_run/EQT_testNLLOC_OBS.hpf NLLOC_OBS ./NLLoc/time/3km0.025 ./NLLoc/loc/ALLPS|g" "./PNR_run/blanketPScindy_nll.in" > "./PNR_run/blanketPScindy_best.in"')

start = datetime.datetime.now()
!./src/NLLoc ./PNR_run/blanketPScindy_best.in
end = datetime.datetime.now()
timing = end - start
print(timing)

NLLoc (NonLinLoc v7.00.00 27Oct2017) 
CONTROL:  MessageFlag: 3  RandomNumSeed: 54321
TRANSFORM  SIMPLE LatOrig 53.775770  LongOrig -2.987780  RotCW 0.000000
LOCSIGNATURE:  Cindy Lim, University of Bristol, PhD project Feb 2021

LOCCOMMENT:  3km x 3 km x 3 km search grid at 0.025km spacing. P phase only.

LOCFILES:  ObsType: NLLOC_OBS  InGrids: ./NLLoc/time/3km0.025.*  OutPut: ./NLLoc/loc/ALLPS.* iSwapBytesOnInput: 0
   Obs File:   0  ./PNR_run/EQT_testNLLOC_OBS.hpf
LOCHYPOUT:  SAVE_NLLOC_ALL SAVE_HYPOINVERSE_Y2000_ARC SAVE_HYPOINV_SUM SAVE_HYPOELL_ALL SAVE_HYPO71_ALL SAVE_HYPOELL_SUM 
LOCSEARCH:  Type: OCT  init_num_cells_x 11  init_num_cells_y 11  init_num_cells_z 11  min_node_size 0.010000  max_num_nodes 20000  num_scatter 5000  use_stations_density 1  stop_on_min_node_size 1  octtreeParams.mean_cell_velocity -1.000000
GRID: {x, y, z}
  Num: {121, 121, 121}
  Orig: {0, 0, 0}
  LenSide: {0.025, 0.025, 0.025}
  Type: PROB_DENSITY
LOCGRID: Save: SAVE
LOCMETHOD:  method: EDT_OT_WT  minDi

In [ ]:
##############################
# NLLOC output --> catalogue
##############################

# path to
origins = []
lats = []
lons = []
depths = []

with open('./NLLoc/loc/ALLPS.sum.grid0.loc.hyp','r') as fi:
  for ln in fi:
    if ln.startswith('GEOGRAPHIC'):
      origins.append(ln[15:42])
      lats.append(float(ln[47:57]))
      lons.append(float(ln[62:72]))
      depths.append(float(ln[78:]))

# loop through and convert origin times to UTCDateTime
for i in range(len(origins)):
  origins[i] = UTCDateTime(origins[i][:4] + origins[i][5:7] + origins[i][8:10] + 'T' + origins[i][12:14] + origins[i][15:17] + "%.4f" % float(origins[i][18:27].zfill(7)))

# create a dataframe of event locations
dfs = pd.DataFrame(list(zip(origins,lats,lons,depths)), columns = ['time', 'lat', 'lon', 'depth'])
# Define the wgs84 and osgb36 projection
wgs84 = pyproj.CRS("EPSG:4326")
osgb = pyproj.CRS("EPSG:27700")

lat = dfs['lat']
lon = dfs['lon']
xx, yy = pyproj.transform(wgs84,osgb,lat,lon)
dfs['xx'] = xx
dfs['yy'] = yy

# # uncomment to save
dfs.to_csv(drive+ '/Catalog/EQT_PNR_catalogue.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [ ]:
dfs

,time,lat,lon,depth,xx,yy
0,2018-12-11T09:26:49.908200Z,53.788886,-2.959864,1.744531,336860.014037,432935.214150
1,2018-12-11T09:38:49.800500Z,53.787582,-2.970382,2.560938,336165.106195,432799.558543
2,2018-12-11T09:42:54.798700Z,53.788426,-2.973628,1.658594,335952.541456,432896.385954
3,2018-12-11T09:45:28.141800Z,53.778531,-2.960261,1.684375,336818.276526,431783.521940


____